In [ ]:
!pip install shapely

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.path as mplPath
#from shapely.geometry import Point
# shapely.geometry.polygon import Polygon

class SailingEnv(gym.Env):
    def __init__(self, max_steps=20_000):
        self.action_space = gym.spaces.Discrete(4) #0=west, 1=north or 2=east, 3=south
        self.minX=self.minY=0#.0
        self.maxX=self.maxY=9#.0
        #self.observation_space = gym.spaces.Box(low=np.array([minX, minY]), high=np.array([maxX, maxY]), dtype=np.float32)
        #self.wind_direction = np.array([1.0, 0.0])
        #self.rudder_angle = 0
        self.x = 0#.0
        self.y = 0#.0
        self.position = np.array([self.x, self.y])
        self.max_steps = max_steps
        self.current_step = 0
        self.target = np.array([8, 9])

    def reset(self):
        #self.wind_direction = np.array([1.0, 0.0])
        #self.rudder_angle = 0
        self.position = np.array([self.x, self.y])
        self.current_step = 0
        return tuple(self.position.astype(np.int64))
    
    def move():
        pass

    def step(self, action):
        #meta to step prepei na epistrepsei:
        #new_state (edw x,y), reward, done , {} 
        assert self.action_space.contains(action)
        self.current_step += 1
        if action==0:
            self.position[0]-=1
        elif action==1:
            self.position[1]+=1
        elif action==2:
            self.position[0]+=1
        else:
            self.position[1]-=1    
            
        #self.position=np.array([1,0])    
        #checks if position after action is contained in our grid
        self.boundaries = [[self.minX-1,self.minY-1], 
                           [self.maxX+1,self.minY-1], 
                           [ self.maxX+1, self.maxY+1], 
                           [self.minX-1,self.maxY+1]]
        self.polygon=mplPath.Path(np.array(self.boundaries))
        in_grid_condition = self.polygon.contains_point(self.position)
        #return in_grid_condition, self.position, self.boundaries
        
#         if self.position[0] < 0  or self.position
        if not in_grid_condition:
            reward=-200
        elif (self.position==self.target).all():
            reward=1
        else:
            reward=-0.5
        

#         reward = 1 if (self.position==self.target).all() else -0.8
#         #reward = np.cos(boat_direction - np.arctan2(self.target[1] - self.position[1], self.target[0] - self.position[0]))
        
        done = self.position[0] >= 10 or self.position[1] >= 10 or self.position[0] < 0 or self.position[1] < 0 or self.current_step >= self.max_steps
#         done = (self.position==self.target).all() 
        
        return tuple(self.position.astype(np.int64)), reward, done, {}

env = SailingEnv()



In [ ]:
done=False
env.reset()
while not done:
    action=env.action_space.sample()
    new_state, reward, done, _ = env.step(action)
    print(f"new state={new_state}, reward={reward}, {env.current_step}-th step, done={done}")


In [400]:
#q_table will have 10X10X4 cells
q_table = np.random.uniform(low=-2, high=0, size=([10,10] + [env.action_space.n]))
LEARNING_RATE = 0.1
DISCOUNT = 0.95

In [401]:
#all_actions=[]
for i in range(5):    
    done=False
    current_state=env.reset()
    ep_reward=0
    while not done:
        action=np.argmax(q_table[current_state]) #best action based on q-values
        #all_actions.append(action)
        new_state, reward, done, _ = env.step(action)
        #Q_max
        Q_max = np.max(q_table[new_state])
        #Q_current
        Q_current = q_table[current_state + (action,)]
        #new_q
        new_q = (1-LEARNING_RATE)*Q_current + LEARNING_RATE*(reward+DISCOUNT*Q_max)
        #update q_table
        q_table[current_state+(action,)]=new_q
        #replace new_state as current_state
        current_state=new_state
        ep_reward+=reward
        #if i==999:
        #print(f"new state={new_state}, reward={reward}, {env.current_step}-th step, done={done}, {i}-th episode")
        print(f"{env.current_step}-th step of {i}-th ep., New Pos={new_state}, RWD={reward}, EP_REW={ep_reward} ")
    print("\n-====================================================================-")


1-th step of 0-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-0.5 
2-th step of 0-th ep., New Pos=(0, 0), RWD=-0.5, EP_REW=-1.0 
3-th step of 0-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-1.5 
4-th step of 0-th ep., New Pos=(0, 0), RWD=-0.5, EP_REW=-2.0 
5-th step of 0-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-2.5 
6-th step of 0-th ep., New Pos=(0, 0), RWD=-0.5, EP_REW=-3.0 
7-th step of 0-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-3.5 
8-th step of 0-th ep., New Pos=(0, 0), RWD=-0.5, EP_REW=-4.0 
9-th step of 0-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-4.5 
10-th step of 0-th ep., New Pos=(-1, 1), RWD=-0.5, EP_REW=-5.0 

-====================================================================-
1-th step of 1-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-0.5 
2-th step of 1-th ep., New Pos=(0, 0), RWD=-0.5, EP_REW=-1.0 
3-th step of 1-th ep., New Pos=(0, 1), RWD=-0.5, EP_REW=-1.5 
4-th step of 1-th ep., New Pos=(0, 0), RWD=-0.5, EP_REW=-2.0 
5-th step of 1-th ep., New Pos=(0, -1), RWD=-200, EP_REW=-

In [ ]:
set(all_actions)

In [ ]:
np.argmax(q_table[current_state])

In [ ]:
q_table[current_state].shape

In [ ]:
current_state

In [ ]:
q_table.shape